### Bước 1: Tạo Park Session và import các thư viện cần thiết.


- Import thư viện càn thiết và thiết lập biến môi trường cho chương trình

In [58]:
import findspark
findspark.init("/home/cuong/Downloads/spark-3.5.5-bin-hadoop3")
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/cuong/Downloads/spark-3.5.5-bin-hadoop3"

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/cuong/Downloads/spark-3.5.5-bin-hadoop3"

- Tạo ParkSession 

In [59]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,next_day,to_date, date_format, expr, lower, sum as spark_sum

spark = SparkSession.builder \
    .appName("RelativePath") \
    .master("local[*]") \
    .config("spark.hadoop.fs.defaultFS", "file:///") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .getOrCreate()

### Bước 2: Đọc file, xem các kiểu dữ liệu có phù hợp chưa, nếu chưa điều chỉnh lại cho đúng. 


In [60]:
df = spark.read.csv("file:///home/cuong/asr.csv", header=True, inferSchema=True)
df.printSchema()


root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: double (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- Unnamed: 22: boolean (nullable = true)



In [61]:
df = df.withColumn("Qty", col("Qty").cast("int"))
df = df.withColumn("order_date", to_date(col('date'), "MM-dd-yy"))
df.printSchema()

root
 |-- index: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel : string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Style: string (nullable = true)
 |-- SKU: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- ASIN: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: double (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- promotion-ids: string (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- Unnamed: 22: boolean (nullable = true)
 |-- order_date: 

### Bước 3: Sắp xếp các record theo thứ tự Date, SKU và filter các record có Status là Shipped.


- Sắp xếp dữ liệu theo ngày và SKU

In [62]:
df_sorted = df.orderBy("order_date", "SKU")
df_sorted.show(10)

+-----+-------------------+--------+--------------------+----------+--------------+------------------+-----+-------------+--------+----+----------+--------------+---+--------+------+-------------+--------------+----------------+------------+--------------------+-----+------------+-----------+----------+
|index|           Order ID|    Date|              Status|Fulfilment|Sales Channel |ship-service-level|Style|          SKU|Category|Size|      ASIN|Courier Status|Qty|currency|Amount|    ship-city|    ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|Unnamed: 22|order_date|
+-----+-------------------+--------+--------------------+----------+--------------+------------------+-----+-------------+--------+----+----------+--------------+---+--------+------+-------------+--------------+----------------+------------+--------------------+-----+------------+-----------+----------+
|48936|404-0332978-0954715|03-31-22|Shipped - Deliver...|  Merchant|     Amazon.in|  

- Lọc ra các cột chứa Status là Shipped

In [63]:
df_sorted = df_sorted.filter(lower(col("Status")).contains("shipped"))
df_sorted.show(10)

+-----+-------------------+--------+--------------------+----------+--------------+------------------+-----+-------------+--------+----+----------+--------------+---+--------+------+-------------+--------------+----------------+------------+--------------------+-----+------------+-----------+----------+
|index|           Order ID|    Date|              Status|Fulfilment|Sales Channel |ship-service-level|Style|          SKU|Category|Size|      ASIN|Courier Status|Qty|currency|Amount|    ship-city|    ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|Unnamed: 22|order_date|
+-----+-------------------+--------+--------------------+----------+--------------+------------------+-----+-------------+--------+----+----------+--------------+---+--------+------+-------------+--------------+----------------+------------+--------------------+-----+------------+-----------+----------+
|48936|404-0332978-0954715|03-31-22|Shipped - Deliver...|  Merchant|     Amazon.in|  

### Bước 4: GroupBy 2 cột date và SKU, tính tổng cột Qty và lưu với tên total_quantity.


- Tính ngày thứ 2 tiếp đến với mỗi ngày tương ứng và gán vào cột `report_date`

In [64]:
result_sorted = df_sorted.withColumn("report_date",next_day(col("order_date"),"Mon"))
result_sorted.show(10)


+-----+-------------------+--------+--------------------+----------+--------------+------------------+-----+-------------+--------+----+----------+--------------+---+--------+------+-------------+--------------+----------------+------------+--------------------+-----+------------+-----------+----------+-----------+
|index|           Order ID|    Date|              Status|Fulfilment|Sales Channel |ship-service-level|Style|          SKU|Category|Size|      ASIN|Courier Status|Qty|currency|Amount|    ship-city|    ship-state|ship-postal-code|ship-country|       promotion-ids|  B2B|fulfilled-by|Unnamed: 22|order_date|report_date|
+-----+-------------------+--------+--------------------+----------+--------------+------------------+-----+-------------+--------+----+----------+--------------+---+--------+------+-------------+--------------+----------------+------------+--------------------+-----+------------+-----------+----------+-----------+
|48936|404-0332978-0954715|03-31-22|Shipped - Del

- Gom nhóm 2 thuộc tính `report_date` và `SKU`

In [65]:
result_sorted = result_sorted.groupBy(['report_date', 'SKU']).agg(spark_sum("Qty").alias("total_quantity"))
result_sorted.printSchema()

root
 |-- report_date: date (nullable = true)
 |-- SKU: string (nullable = true)
 |-- total_quantity: long (nullable = true)



### Bước 5: Format lại dữ liệu theo yêu cầu và ghi ra file.


- Chỉnh định dạng ngày cho đúng yêu cầu

In [66]:
result_sorted = result_sorted.withColumn("report_date", to_date(col('report_date'), "dd-MM-yyyy"))
result_sorted.show()

+-----------+-----------------+--------------+
|report_date|              SKU|total_quantity|
+-----------+-----------------+--------------+
| 2022-05-02|  NW032-ST-CP-XXL|             1|
| 2022-05-02|       J0334-TP-L|             5|
| 2022-05-02|       J0338-DR-L|             7|
| 2022-05-02|       J0157-DR-S|             5|
| 2022-05-02|   NW030-TP-PJ-XL|             5|
| 2022-05-02|   SET184-KR-PP-M|             4|
| 2022-05-02|      J0337-TP-XL|             2|
| 2022-05-02|SET402-KR-NP-XXXL|             3|
| 2022-05-02|  JNE3804-KR-XXXL|             2|
| 2022-05-02| SET198-KR-NP-A-M|             1|
| 2022-05-02|      J0377-SKD-L|             2|
| 2022-04-25|     JNE3291-KR-M|            26|
| 2022-04-25|      J0344-TP-XL|             6|
| 2022-04-25|     JNE3675-TU-M|             2|
| 2022-04-25|  JNE3543-KR-XXXL|             2|
| 2022-04-25| SET304-KR-DPT-XL|             3|
| 2022-04-25|   SET038-KR-PP-M|             2|
| 2022-04-25|   SET320-KR-NP-M|             1|
| 2022-04-25|

- Lại theo ngày và SKU như yêu cầu của đề

In [71]:
result_sorted = result_sorted.orderBy('report_date', 'SKU')
result_sorted = result_sorted.withColumn("report_date", date_format("report_date", "dd-MM-yyyy"))


- Sửa cột `SKU` thành `sku` 

In [72]:
result_sorted = result_sorted.withColumnRenamed("SKU", 'sku')
result_sorted.show(10)

+-----------+---------------+--------------+
|report_date|            sku|total_quantity|
+-----------+---------------+--------------+
| 04-04-2022| AN209-BIEGE-XL|             1|
| 04-04-2022|BL003-50BLACK-B|             1|
| 04-04-2022|  BL009-61BLACK|             3|
| 04-04-2022|   BL015-63PINK|             1|
| 04-04-2022| BL023-74PINK-B|             1|
| 04-04-2022|  BL035-161GOLD|             4|
| 04-04-2022|BL057-65BLACK-A|             3|
| 04-04-2022|        BL099-L|             1|
| 04-04-2022|        BL102-L|             1|
| 04-04-2022|       BL103-XS|             1|
+-----------+---------------+--------------+
only showing top 10 rows



- Ghi kết quả ra file

In [73]:
result_sorted.coalesce(1).write.csv('file:///home/cuong/CQ05_Lab02/src/Task_2.2/output', header=True, mode='overwrite')